In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.distance import pdist
from sklearn import metrics
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics.pairwise import linear_kernel
from sklearn.model_selection import cross_val_score
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.kernel_ridge import KernelRidge

from sklearn import svm

import dionysus as d
import sklearn_tda as tda

from biomarker.data_collection import *

Cython not found--WassersteinDistance not available
Gudhi not found--GraphInducedComplex not available
data_collection


In [2]:
EXCLUDE_KEYS = [206, 205, 184, 183, 82, 81, 45]
TEST_KEYS = [217, 216]#, 215, 214, 213, 212, 211, 210, 209]

In [5]:
excel = parse_master_file(exclude_keys=EXCLUDE_KEYS)#.reset_index(drop=True)
keys = excel['Key']
L = get_filename_list(excel['Associated data'])

[3, 9]


In [51]:
parse_x1(L[0])

,Center Number,Atomic Number,X,Y,Z
0,1,6,0.000059,0.043700,1.801291
1,2,6,-1.258000,-0.689764,1.301632
2,3,6,-1.279643,-0.732625,-0.243112
3,4,6,-1.251487,0.717669,-0.776895
4,5,6,0.000657,1.472170,-0.291885
5,6,6,0.000697,1.484036,1.255045
6,7,6,1.252129,0.716583,-0.776911
7,8,6,1.278969,-0.733750,-0.243145
8,9,6,-0.000668,-1.452102,-0.727830
9,10,6,1.257432,-0.690900,1.301588


In [50]:
parse_x3(L[0])

,1,2,3,4,5,6,7,8,9,10,...,25,26,27,28,29,30,31,32,33,34
0,6.413313,-0.150839,-0.244460,-0.025075,0.501949,-0.686008,-0.024930,-0.244566,0.295831,-0.150821,...,-0.022480,-0.054230,0.000233,0.000231,-0.004561,-0.007188,-0.000916,-0.000916,-0.007186,-0.004561
1,-0.150839,7.205926,-0.346513,-0.452637,0.407716,-0.122080,-0.066618,0.400107,-0.380641,-0.315474,...,0.005476,0.002278,0.000455,0.001591,0.002682,0.017368,-0.005597,0.059006,-0.053843,-0.051388
2,-0.244460,-0.346513,8.007744,-0.941786,0.043573,0.052109,0.673952,-0.438020,-1.227049,0.400077,...,-0.007152,0.002471,0.009321,-0.004937,-0.006519,-0.010680,-0.007432,0.010774,-0.022169,-0.041913
3,-0.025075,-0.452637,-0.941786,10.019569,-1.433266,0.372890,-1.524587,0.674012,-0.679888,-0.066505,...,0.036189,0.016137,-0.137048,0.056395,-0.005976,-0.005912,0.021493,-0.074514,0.071119,-0.052295
4,0.501949,0.407716,0.043573,-1.433266,9.205985,-1.826607,-1.433756,0.043629,0.704421,0.407918,...,0.029518,-0.017393,0.059528,0.059515,-0.008913,-0.004766,0.009354,0.009332,-0.004758,-0.008900
5,-0.686008,-0.122080,0.052109,0.372890,-1.826607,7.974678,0.373237,0.052149,-0.661438,-0.122211,...,-0.045979,0.015671,0.009563,0.009566,0.014976,-0.013221,-0.011950,-0.011940,-0.013222,0.014960
6,-0.024930,-0.066618,0.673952,-1.524587,-1.433756,0.373237,10.020272,-0.942104,-0.680015,-0.452978,...,-0.066254,0.039252,0.056385,-0.137054,-0.052274,0.071117,-0.074534,0.021497,-0.005922,-0.005973
7,-0.244566,0.400107,-0.438020,0.674012,0.043629,0.052149,-0.942104,8.008120,-1.227010,-0.346394,...,-0.060184,-0.060395,-0.004936,0.009324,-0.041930,-0.022183,0.010786,-0.007436,-0.010675,-0.006521
8,0.295831,-0.380641,-1.227049,-0.679888,0.704421,-0.661438,-0.680015,-1.227010,10.420519,-0.380791,...,0.077208,-0.016725,-0.002201,-0.002201,0.084577,-0.053226,-0.059993,-0.059971,-0.053250,0.084580
9,-0.150821,-0.315474,0.400077,-0.066505,0.407918,-0.122211,-0.452978,-0.346394,-0.380791,7.206149,...,0.360491,0.375673,0.001591,0.000454,-0.051402,-0.053858,0.059026,-0.005599,0.017374,0.002680


In [52]:
parse_x4(L[0])

,Val1,Val2,Val3
0,0.000059,0.043700,1.801291
1,-1.258000,-0.689764,1.301632
2,-1.279643,-0.732625,-0.243112
3,-1.251487,0.717669,-0.776895
4,0.000657,1.472170,-0.291885
5,0.000697,1.484036,1.255045
6,1.252129,0.716583,-0.776911
7,1.278969,-0.733750,-0.243145
8,-0.000668,-1.452102,-0.727830
9,1.257432,-0.690900,1.301588


In [45]:
parse_x5(L[0])

,Electric Potential,X,Y,Z
0,-14.773684,0.000000,-0.000174,0.000070
1,-14.785659,0.001544,0.000822,-0.001676
2,-14.765329,-0.000827,-0.000585,-0.000252
3,-14.788648,0.001855,-0.001159,0.000577
4,-14.742100,-0.000001,-0.004339,0.001995
5,-14.786371,-0.000001,-0.001873,-0.002278
6,-14.788648,-0.001857,-0.001158,0.000577
7,-14.765331,0.000825,-0.000587,-0.000252
8,-14.787512,0.000001,0.002112,0.001362
9,-14.785660,-0.001544,0.000824,-0.001675


In [63]:
parse_x6(L[-3])

,XX,YY,ZZ,XY,XZ,YZ
1,-812.734912,-812.809481,-812.576571,0.000010,-0.013038,0.000000
2,-500.756464,-500.980212,-501.274034,0.167747,-0.003398,-0.004967
3,-500.734865,-500.706179,-501.210571,-0.015331,-0.000829,-0.000313
4,-500.751080,-500.749418,-501.238974,-0.000002,-0.000765,0.000000
5,-500.734882,-500.706157,-501.210560,0.015332,-0.000827,0.000311
6,-500.756490,-500.980193,-501.274095,-0.167726,-0.003395,0.004963
7,-500.751081,-500.749418,-501.238975,0.000004,-0.000758,0.000000
8,-500.734884,-500.706154,-501.210555,-0.015331,-0.000811,-0.000311
9,-500.755193,-500.853234,-501.331229,0.069156,-0.002716,-0.000122
10,-500.755192,-500.853235,-501.331227,-0.069155,-0.002718,0.000113
